# PostgreSQL Transaction Isolation Levels in Python
This notebook demonstrates the 4 standard isolation levels in PostgreSQL:
- READ UNCOMMITTED (emulated as READ COMMITTED in PostgreSQL)
- READ COMMITTED
- REPEATABLE READ
- SERIALIZABLE

In [1]:
import asyncio
from enum import Enum
from typing import Literal

import nest_asyncio
from sqlalchemy import delete, select, text, update

from db.postgre import async_engine, async_session
from models.users import Base, User

nest_asyncio.apply()

In [2]:
class IsolationLevels(str, Enum):
    READ_UNCOMMITTED = "READ UNCOMMITTED"
    READ_COMMITTED = "READ COMMITTED"
    REPEATABLE_READ = "REPEATABLE READ"
    SERIALIZABLE = "SERIALIZABLE"

## 1. READ UNCOMMITTED
`READ UNCOMMITTED` is the lowest isolation level in the SQL standard. It allows a transaction to read data that has been modified by other transactions but not yet committed.

This leads to `dirty reads` — transactions can see intermediate, uncommitted changes from others.

🔒 PostgreSQL does not allow dirty reads, even if you set `READ UNCOMMITTED` — it behaves like `READ COMMITTED`.

## 2. READ COMMITTED (default in PostgreSQL)

Each query sees only committed data as of the start of the query. It allows:
- `non-repeatable reads` — occurs when a transaction reads **the same row** twice and gets different values;
- `phantom reads` — occurs when a transaction executes the same query (returning **multiple rows** based on a condition) twice and gets a different **set of rows**.

### Example

Session 1:
```sql
BEGIN;
SELECT surname FROM users WHERE unique_code = 'abc123';
-- Result: 'Smith'
```

Session 2:
```sql
BEGIN;
UPDATE users SET surname = 'Doe' WHERE unique_code = 'abc123';
COMMIT;
```

Back to Session 1:
```sql
SELECT surname FROM users WHERE unique_code = 'abc123';
-- Result: 'Doe'
COMMIT;
```


In [3]:
async def setup() -> None:
    """Setup the database and create a test user."""
    async with async_engine.begin() as conn:
        await conn.run_sync(Base.metadata.create_all)

    async with async_session() as session:
        await session.begin()
        await session.execute(delete(User))
        session.add(User(name="Alice", surname="Smith", unique_code="abc123"))
        await session.commit()

In [4]:
async def session1(
    isolation_level: Literal[IsolationLevels.READ_UNCOMMITTED,
                             IsolationLevels.READ_COMMITTED,
                             IsolationLevels.REPEATABLE_READ,
                             IsolationLevels.SERIALIZABLE,
                            ],
) -> None:
    """Session 1: Reads the surname of Alice.
    
    Args:
        isolation_level: The isolation level to use for the session.
    """
    session_name = "Session 1"

    async with async_session() as session:
        await session.begin()
        await session.connection(execution_options={"isolation_level": isolation_level})
        result = await session.execute(
            select(User.surname).where(User.unique_code == "abc123")
        )
        print(f"{session_name}: Alice surname is {result.scalar_one()}")
        
        await asyncio.sleep(10)
        result = await session.execute(
            select(User.surname).where(User.unique_code == "abc123")
        )
        print(f"{session_name}: Alice surname is {result.scalar_one()}")
        await session.commit()
        print(f"{session_name}: Session committed...")


async def session2(
    isolation_level: Literal[IsolationLevels.READ_UNCOMMITTED,
                             IsolationLevels.READ_COMMITTED,
                             IsolationLevels.REPEATABLE_READ,
                             IsolationLevels.SERIALIZABLE,
                            ],
) -> None:
    """Session 2: Updates the surname of Alice."""
    session_name = "Session 2"
    new_surname = "Doe"

    async with async_session() as session:
        await session.begin()
        await session.connection(execution_options={"isolation_level": isolation_level})
        await asyncio.sleep(5)
        await session.execute(
            update(User).where(User.unique_code == "abc123").values(surname=new_surname)
        )
        print(f"{session_name}: Alice surname was updated to '{new_surname}'...")
        await session.commit()
        print(f"{session_name}: Session committed...")


await setup()
_ = await asyncio.gather(
    session1(isolation_level=IsolationLevels.READ_COMMITTED),
    session2(isolation_level=IsolationLevels.READ_COMMITTED),
    )

Session 1: Alice surname is Smith
Session 2: Alice surname was updated to 'Doe'...
Session 2: Session committed...
Session 1: Alice surname is Doe
Session 1: Session committed...


## 3. REPEATABLE READ

All reads within the transaction see the same snapshot (taken at `BEGIN`).

Prevents `dirty reads` and `non-repeatable reads`, but theoretically allows `phantom rows`. 

PostgreSQL implements `REPEATABLE READ` as `snapshot isolation`, which means:
- Each transaction receives a snapshot of the database as it existed at the start of the transaction.
- As a result, all repeated `SELECT` queries within that transaction see the same set of rows, even if other transactions insert, update, or delete data afterward.

🔒 Therefore, `phantom reads` are not possible under `REPEATABLE READ` in PostgreSQL.

### Example

Session 1:
```sql
BEGIN;
SELECT surname FROM users WHERE unique_code = 'abc123';
-- Result: 'Smith'
```

Session 2:
```sql
BEGIN;
UPDATE users SET surname = 'Doe' WHERE unique_code = 'abc123';
COMMIT;
```

Back to Session 1:
```sql
SELECT surname FROM users WHERE unique_code = 'abc123';
-- Result: still 'Smith'
COMMIT;
```

In [5]:
await setup()
_ = await asyncio.gather(
    session1(isolation_level=IsolationLevels.REPEATABLE_READ),
    session2(isolation_level=IsolationLevels.REPEATABLE_READ),
    )

Session 1: Alice surname is Smith
Session 2: Alice surname was updated to 'Doe'...
Session 2: Session committed...
Session 1: Alice surname is Smith
Session 1: Session committed...


## 4. Serializable

Full isolation, PostgreSQL will abort one of the transactions if a conflict is detected. Can say that it same as `REPEATABLE READ`, but with extra checks for serialization safety. A serialization failure (`SQLSTATE 40001`) in PostgreSQL typically occurs only when there's a potential `write-write` or `read-write` conflict that can't be resolved while preserving the illusion of serial execution.

In [ ]:
# TODO: rewrite sessions in such way to produce serialization failure

await setup()
_ = await asyncio.gather(
    session1(isolation_level=IsolationLevels.SERIALIZABLE),
    session2(isolation_level=IsolationLevels.SERIALIZABLE),
    )

Session 1: Alice surname is Smith
Session 2: Alice surname was updated to 'Doe'...
Session 2: Session committed...
Session 1: Alice surname is Smith
Session 1: Session committed...


| Isolation level      | Dirty Reads | Non-Repeatable Reads | Phantom Reads                  |
| -------------------- | ----------- | -------------------- | ------------------------------ |
| **READ UNCOMMITTED** | ✅         | ✅                   | ✅                            |
| **READ COMMITTED**   | ❌         | ✅                   | ✅                            |
| **REPEATABLE READ**  | ❌         | ❌                   | *(in SQL: ✅, in PostgreSQL: ❌)* |
| **SERIALIZABLE**     | ❌         | ❌                   | ❌                            |
